In [2]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
pyodbc.drivers()

['SQL Server',
 'PostgreSQL ANSI(x64)',
 'PostgreSQL Unicode(x64)',
 'Amazon Redshift (x64)',
 'MySQL ODBC 8.0 ANSI Driver',
 'MySQL ODBC 8.0 Unicode Driver',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server']

In [3]:
cnxn = pyodbc.connect(driver='{SQL Server}', host='LAPTOP-EGF03VVI', database='Radian',
                      trusted_connection='tcon')

In [4]:
cursor = cnxn.cursor()


In [5]:
query = ''' 
SELECT 
DISTINCT b.[Category_Desc], b.[item_number], c.[WEEK_SAT],c.[Sales], c.[NetCost],
c.[Markdown], c.[Quantity], c.STORE, c.[Weight]
FROM [dbo].[CaseItemMaster] as b
INNER JOIN [dbo].[CaseTransactions] as c
ON b.upc = c.UPC
INNER JOIN [dbo].[CaseStoreMaster] as a
ON c.STORE = a.Store_Value
WHERE a.Active = 'Y'AND
NOT (a.District_Zone IS NULL 
OR b.brandname IS NULL
OR b.[Category_Desc] IS NULL) AND 
b.item_status < '9'
AND 
(c.Quantity > 0
AND c.[Sales] > 0
AND c.[NetCost] > 0
AND c.[Markdown] > 0
AND c.[Weight] > 0)

'''



Loading a data

In [6]:
data = pd.read_sql(query, cnxn)

In [7]:
data.head()

,Category_Desc,item_number,WEEK_SAT,Sales,NetCost,Markdown,Quantity,STORE,Weight
0,SEAFOOD - FRESH BULK,651527,2014-12-06,43.35,18.270,13.03,8,40.0,4.34
1,MEAT -BEEF CERTIFIED TENDER,336138,2014-12-06,189.73,102.420,65.91,17,18.0,14.61
2,MEAT SERVICE - BEEF,542810,2014-10-11,81.04,43.740,13.53,8,58.0,13.53
3,MEAT SERVICE - BEEF,542808,2015-02-21,3.15,1.701,0.55,1,37.0,0.50
4,MEAT - BEEF ANGUS,493069,2015-03-21,377.89,204.039,125.79,26,1.0,62.05


# Data Cleaning

In [8]:
#Rename the columns to make it consistent

data.rename(columns = {'Category_Desc':'Categories','item_number':'Item_number','WEEK_SAT':'Week_Sat','STORE':'Store'}, inplace = True)

In [9]:
data.head(5)

,Categories,Item_number,Week_Sat,Sales,NetCost,Markdown,Quantity,Store,Weight
0,SEAFOOD - FRESH BULK,651527,2014-12-06,43.35,18.270,13.03,8,40.0,4.34
1,MEAT -BEEF CERTIFIED TENDER,336138,2014-12-06,189.73,102.420,65.91,17,18.0,14.61
2,MEAT SERVICE - BEEF,542810,2014-10-11,81.04,43.740,13.53,8,58.0,13.53
3,MEAT SERVICE - BEEF,542808,2015-02-21,3.15,1.701,0.55,1,37.0,0.50
4,MEAT - BEEF ANGUS,493069,2015-03-21,377.89,204.039,125.79,26,1.0,62.05


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53826 entries, 0 to 53825
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Categories   53826 non-null  object        
 1   Item_number  53826 non-null  object        
 2   Week_Sat     53826 non-null  datetime64[ns]
 3   Sales        53826 non-null  float64       
 4   NetCost      53826 non-null  float64       
 5   Markdown     53826 non-null  float64       
 6   Quantity     53826 non-null  int64         
 7   Store        53826 non-null  float64       
 8   Weight       53826 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(2)
memory usage: 3.7+ MB


In [11]:
data.isnull().sum()

Categories     0
Item_number    0
Week_Sat       0
Sales          0
NetCost        0
Markdown       0
Quantity       0
Store          0
Weight         0
dtype: int64

In [12]:
#Extracting the years and months from Week_Sat datetime index for future use
data['Week_Sat'] = pd.to_datetime(data['Week_Sat'])
data['Year'] = data['Week_Sat'].dt.year


# Exploratory Data Analysis

Which categories have highest markdown rate

In [13]:
new_df =  pd.DataFrame(data[['Markdown','Categories','Year']]).copy()

In [14]:
Best_Categories = new_df.groupby(['Categories','Year'])['Markdown'].sum().reset_index()

In [15]:
#Top Categories that have highest markdown rate 
Best_Categories.sort_values(['Markdown'],ascending = False).head()

,Categories,Year,Markdown
31,MEAT - FRESH PORK,2015,521590.13
23,MEAT - BEEF GROUND,2015,475853.07
27,MEAT - FRESH CHICKEN,2015,383155.15
19,MEAT - BEEF ANGUS,2015,307586.31
38,MEAT - SMOKED,2015,252372.86


Which Categories are declining more than 30% year over year

In [ ]:
declining_df = pd.DataFrame(data[['Categories','Year','Sales']]).copy()

In [ ]:
decliningrate = pd.DataFrame(declining_df.groupby(['Year','Categories']).agg({'Sales':'sum'}))
#Sales_pcts = decliningrate.groupby(level=1).apply(lambda x:
#                                                 100 * x / float(x.sum()))            

decliningrate.sort_values(by = ['Year','Sales'], ascending = False).round(1)

In [ ]:
conditional = Sales_pcts[Sales_pcts['Sales'] < 30.0]
conditional

Biggest Categories by Sales,Volume and Margin

In [ ]:
Net_Sales = data['Sales']- data['Markdown']
Net_Profit = Net_Sales - data['NetCost']
data['Margin'] = Net_Profit / Net_Sales

In [ ]:
data

In [ ]:
sales = pd.DataFrame(data[['Categories','Sales']])
cat_sales = pd.DataFrame(sales.groupby('Categories')['Sales'].sum())
cat_sales.sort_values(['Sales'],ascending =False).head()

In [ ]:
weight = pd.DataFrame(data[['Categories','Weight']])
cat_weight = pd.DataFrame(sales.groupby('Categories')['Weight'].sum())
cat_weight.sort_values(['Weight'],ascending =False).head()

In [ ]:
margin = pd.DataFrame(data[['Categories','Margin']])
cat_margin = pd.DataFrame(margin.groupby('Categories')['Margin'].sum())
cat_margin.sort_values(['Margin'],ascending =False).head()

What are the best and worst 10 stores based on Sales (Year over Year – henceforth YoY)

In [ ]:
#Creating a new dataframe for topstores 
Topstores = pd.DataFrame(data.groupby(['Year','Store']).sum()['Sales']).reset_index().sort_values('Sales', ascending = False)
Topstores['Store'] = Topstores['Store'].astype('category')

In [ ]:
#Top 10 best
Topstores['DifferenceYoY'] = pd.DataFrame(Topstores['Sales'].diff())
Topstores[Topstores['Year']==2015]

In [ ]:
Topstores['DifferenceYoY'] = pd.DataFrame(Topstores['Sales'].diff())
Topstores[Topstores['Year']==2014]

Rank all Stores based on Sales, Volume, Margin , Sales Yoy , VolumeYoy, Marging Yoy

In [ ]:
Rank_Tables =pd.DataFrame(data[['Year','Store','Sales','Weight','Margin']]).copy()
Rank_Tables['MarginYoY'] = Rank_Tables['Margin'].diff()
Rank_Tables['WeightYoY']= Rank_Tables['Weight'].diff()
Rank_Tables['SalesYoY'] = Rank_Tables['Sales'].diff()

In [ ]:
cols = ['Sales', 'Weight', 'Margin', 'MarginYoY', 'WeightYoY',
       'SalesYoY']

stores_rank = pd.pivot_table(Rank_Tables, index = 'Store',columns = 'Year',aggfunc = 'mean', values = cols)
stores_rank

In [ ]:
stores_rank.rank(ascending = False)